In [1]:
import os
from time import time
from tqdm import tqdm
import numpy

import torch
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.transforms import transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import opendatasets as od
import pandas as pd
od.download('https://www.kaggle.com/datasets/rtlmhjbn/ip02-dataset/code?select=classification')

Skipping, found downloaded files in ".\ip02-dataset" (use force=True to force download)


In [5]:
f = open('ip02-dataset/classes.txt')
label = []
name = []
for line in f.readlines():
    label.append(int(line.split()[0]))
    name.append(' '.join(line.split()[1:]))
classes = pd.DataFrame([label, name]).T
classes.columns = ['label','name']
classes

,label,name
0,1,rice leaf roller
1,2,rice leaf caterpillar
2,3,paddy stem maggot
3,4,asiatic rice borer
4,5,yellow rice borer
...,...,...
97,98,Chlumetia transversa
98,99,Mango flat beak leafhopper
99,100,Rhytidodera bowrinii white
100,101,Sternochetus frigidus


In [6]:
data_path = 'ip02-dataset/classification'
stuff = os.listdir(data_path)
print(len(stuff))

3


In [8]:
train_df = pd.read_csv('ip02-dataset/train.txt', sep=' ', header=None, engine='python')
train_df.columns = ['image_path', 'label']

test_df = pd.read_csv('ip02-dataset/test.txt', sep=' ', header=None, engine='python')
test_df.columns = ['image_path', 'label']

val_df = pd.read_csv('ip02-dataset/val.txt', sep=' ', header=None, engine='python')
val_df.columns = ['image_path', 'label']

print(len(train_df))
print(len(test_df))
print(len(val_df))
train_df.head()

45095
22619
7508


,image_path,label
0,00002.jpg,0
1,00003.jpg,0
2,00005.jpg,0
3,00006.jpg,0
4,00008.jpg,0


In [9]:
tfm = transforms.Compose([transforms.ToTensor(),
                                     transforms.RandomVerticalFlip(),
                                     transforms.RandomAdjustSharpness(sharpness_factor=1.5),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ColorJitter(),
                                     transforms.Resize((224,224))])

In [21]:
train_ds = train_df['image_path'].tolist()
test_ds = test_df['image_path'].tolist()
val_ds = val_df['image_path'].tolist()

#current_directory = os.getcwd()
#print(current_directory)

# I cant find it for some reason :/
train_root = '/ip02-datase/train.txt'
test_root= '/ip02-dataset/test.txt'
val_root = 'ip02-dataset/val.txt'

train_ds = ImageFolder(train_root, transform=tfm)
test_ds = ImageFolder(test_root, transform=tfm)
valid_ds = ImageFolder(val_root, transform=tfm)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/ip02-datase/train.txt'

In [14]:
LEN_TRAIN = len(train_ds)
LEN_TEST = len(test_ds)
print(LEN_TRAIN, LEN_TEST)
print(train_ds.class_to_idx)

C:\Users\Areen\Documents\GitHub\Group033-SP23


In [ ]:
train_loader = DataLoader(train_ds, batch_size = 30, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=30, shuffle = True)

In [ ]:
model = resnet18(pretrained=True)

#unsure of the # for in and out features
model.fc = Linear(in_features= , out_features=)
model = model.to(device)


In [ ]:
optimiser = Adam(model.parameters(), lr=3e-4, weight_decay=0.0001)
loss_fn = CrossEntropyLoss()

In [ ]:
for epoch in range(3):
    start = time()
    
    tr_acc = 0
    test_acc = 0
    
    model.train()
    
    with tqdm(train_loader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            optimiser.zero_grad()
            
            xtrain = xtrain.to(device)
            train_prob = model(xtrain)
            train_prob = train_prob.cpu()
            
            loss = loss_fn(train_prob, ytrain)
            loss.backward()
            optimiser.step()
            
            # Training ends
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / LEN_TRAIN
    
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        for xtest, ytest in test_loader:
            xtest = xtest.to(device)
            test_prob = model(xtest)
            test_prob = test_prob.cpu()
            
            test_pred = torch.max(test_prob,1).indices
            test_acc += int(torch.sum(test_pred == ytest))
            
        ep_test_acc = test_acc / LEN_TEST
    
    end = time()
    duration = (end - start) / 60
    
    print(f"Epoch: {epoch}, Time: {duration}, Loss: {loss}\nTrain_acc: {ep_tr_acc}, Test_acc: {ep_test_acc}")